### Word Group In Youtube Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "Italian"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = Italian
Lang Id = 7
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/3-Word Group In Youtube Sentence Firebase"

#Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
app = firebase_admin.initialize_app(cred)

In [7]:
fire_db = firestore.client()

#### Read Data

In [8]:
#parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Data/1-Word Group In Youtube Sentence"
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Result/2-Word Group Prepare For Youtube Tool"

In [9]:
#df_28 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx")
df_28 = df_28.head(200)
df_28

,search_string,start_time,end_time,sentence,video_id,video_url
0,che sono una cosa che non lo,206,208,che sono una cosa che non lo sa più,iBdSB7CjB0A,https://www.youtube.com/watch?v=iBdSB7CjB0A&t=...
1,è una cosa che non si,3373,3375,è una cosa che non si vede,0fjmn2TpzH4,https://www.youtube.com/watch?v=0fjmn2TpzH4&t=...
2,che non è una cosa da,322,323,che non è una cosa da poco,X9Z9iwK4whQ,https://www.youtube.com/watch?v=X9Z9iwK4whQ&t=...
3,con che cosa con una,80,83,con che cosa con una frusta,ZsnF4gwcYaI,https://www.youtube.com/watch?v=ZsnF4gwcYaI&t=80s
4,e non è una cosa da,210,212,e non è una cosa da poco,Efb1ZbVxHik,https://www.youtube.com/watch?v=Efb1ZbVxHik&t=...
...,...,...,...,...,...,...
195,di cosa,158,160,di cosa,B2UU2a4kqXU,https://www.youtube.com/watch?v=B2UU2a4kqXU&t=...
196,sono le,666,670,sono le,1hQksaX5vb8,https://www.youtube.com/watch?v=1hQksaX5vb8&t=...
197,e per l,167,170,e per l,JrZx_MgXZQc,https://www.youtube.com/watch?v=JrZx_MgXZQc&t=...
198,sono le,30,32,sono le,3CIIzr6Ohv0,https://www.youtube.com/watch?v=3CIIzr6Ohv0&t=30s


In [10]:
#df_200 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx")
df_200 = df_200.head(200)
df_200

,search_string,start_time,end_time,sentence,video_id,video_url
0,vedere meglio e siamo qui allora quello che no...,1661,1669,vedere meglio e siamo qui allora quello che no...,GH1Zq1GarM4,https://www.youtube.com/watch?v=GH1Zq1GarM4&t=...
1,come tu hai fatto sì io ho fatto questa cosa h...,1372,1387,come tu hai fatto sì io ho fatto questa cosa h...,_9o21e4PwZs,https://www.youtube.com/watch?v=_9o21e4PwZs&t=...
2,non è quello ok adesso sto a casa di qualcuno ...,36,41,probabilmente non è quello ok adesso sto a cas...,9SesLBahFjc,https://www.youtube.com/watch?v=9SesLBahFjc&t=36s
3,quello che siamo perché lo siamo già stato pri...,20,25,a capire quello che siamo perché lo siamo già ...,L2CyvarDzi8,https://www.youtube.com/watch?v=L2CyvarDzi8&t=20s
4,che tutti noi prima o poi o abbiamo fatto o ab...,107,114,have che tutti noi prima o poi o abbiamo fatto...,sHTcserYi68,https://www.youtube.com/watch?v=sHTcserYi68&t=...
...,...,...,...,...,...,...
195,quello dopo va bene andiamo a farlo da te ne,1095,1104,quello dopo va bene andiamo a farlo da te ne f...,b0PpruIdqUM,https://www.youtube.com/watch?v=b0PpruIdqUM&t=...
196,perché noi siamo adesso io prima ho detto la,218,222,perché noi siamo adesso io prima ho detto la roma,lQAdxD15TA0,https://www.youtube.com/watch?v=lQAdxD15TA0&t=...
197,quindi adesso questa è la prima e lavoro una,1487,1491,quindi adesso questa è la prima e lavoro una m...,g4Fjlu2sWxM,https://www.youtube.com/watch?v=g4Fjlu2sWxM&t=...
198,po tutti non so voi ma credo di sì ma quando,15,20,po tutti non so voi ma credo di sì ma quando ero,oaqjdcwLSaI,https://www.youtube.com/watch?v=oaqjdcwLSaI&t=15s


In [11]:
#df_1000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx")
df_1000 = df_1000.head(200)
df_1000

,search_string,start_time,end_time,sentence,video_id,video_url
0,tu hai sentito parlare di questa cosa semplice...,1469,1475,tu hai sentito parlare di questa cosa semplice...,aFjEiPXtL6w,https://www.youtube.com/watch?v=aFjEiPXtL6w&t=...
1,grazie per essere stato o per essere stata qui...,455,463,grazie per essere stato o per essere stata qui...,eljZW_3E79g,https://www.youtube.com/watch?v=eljZW_3E79g&t=...
2,perché questo video è stato molto più difficil...,58,64,perché questo video è stato molto più difficil...,brR9KopWpt4,https://www.youtube.com/watch?v=brR9KopWpt4&t=58s
3,grazie per essere stato o per essere stata qui...,494,501,grazie per essere stato o per essere stata qui...,3ArISj4cyPo,https://www.youtube.com/watch?v=3ArISj4cyPo&t=...
4,una domanda semplice questa è la nostra ultima...,637,647,buonasera corrado una domanda semplice questa ...,HAqfb5O8LaY,https://www.youtube.com/watch?v=HAqfb5O8LaY&t=...
...,...,...,...,...,...,...
195,comunque vi faccio vedere una cosa guardate ve...,543,549,comunque vi faccio vedere una cosa guardate ve...,EzKhZkxBG9g,https://www.youtube.com/watch?v=EzKhZkxBG9g&t=...
196,questa situazione devo sapere io cosa va megli...,443,449,questa situazione devo sapere io cosa va megli...,bZiCGouQajs,https://www.youtube.com/watch?v=bZiCGouQajs&t=...
197,con se stessa e vivere al meglio e quindi vuol...,548,556,con se stessa e vivere al meglio e quindi vuol...,h7vP50AwR3Q,https://www.youtube.com/watch?v=h7vP50AwR3Q&t=...
198,non mi voleva vedere io ho detto fatto la stes...,4014,4021,non mi voleva vedere io ho detto fatto la stes...,VF3EauKu290,https://www.youtube.com/watch?v=VF3EauKu290&t=...


In [12]:
#df_5000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx")
df_5000 = df_5000.head(200)
df_5000

,search_string,start_time,end_time,sentence,video_id,video_url
0,ciao come stai vuol dire che del tuo umore imp...,17,38,ciao come stai vuol dire che del tuo umore imp...,8PQ71A2qPRk,https://www.youtube.com/watch?v=8PQ71A2qPRk&t=17s
1,comunque ti ringrazio per essere stato in mia ...,800,812,comunque ti ringrazio per essere stato in mia ...,uB0AorgbwTc,https://www.youtube.com/watch?v=uB0AorgbwTc&t=...
2,andati a letto i bambini dopo un ora stavano t...,250,261,andati a letto i bambini dopo un ora stavano t...,CK71slKyiXU,https://www.youtube.com/watch?v=CK71slKyiXU&t=...
3,per cui in qualche modo possiamo vedere se riu...,820,830,per cui in qualche modo possiamo vedere se riu...,Ek0vTrfaxX0,https://www.youtube.com/watch?v=Ek0vTrfaxX0&t=...
4,seguendo però sempre la sensazione che si ha n...,739,748,seguendo però sempre la sensazione che si ha n...,uB0AorgbwTc,https://www.youtube.com/watch?v=uB0AorgbwTc&t=...
...,...,...,...,...,...,...
195,ancora attuale però non da parte mia ma dalla ...,980,989,ancora attuale però non da parte mia ma dalla ...,ZFQDdaWGikA,https://www.youtube.com/watch?v=ZFQDdaWGikA&t=...
196,velocemente questo spiegato proprio in modo fa...,551,557,velocemente questo spiegato proprio in modo fa...,Jdvh43Al_Q0,https://www.youtube.com/watch?v=Jdvh43Al_Q0&t=...
197,ma questo se lo avete ordinato subito se aspet...,1373,1382,ma questo se lo avete ordinato subito se aspet...,0fHCy7KHXlY,https://www.youtube.com/watch?v=0fHCy7KHXlY&t=...
198,difficile da farlo girare velocemente è un po ...,532,540,difficile da farlo girare velocemente è un po ...,Gy1e_bHLlwY,https://www.youtube.com/watch?v=Gy1e_bHLlwY&t=...


In [13]:
#df_10000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx")
df_10000 = df_10000.head(200)
df_10000

,search_string,start_time,end_time,sentence,video_id,video_url
0,quindi venivo qui due volte a settimana il sab...,294,305,quindi venivo qui due volte a settimana il sab...,miyE4qxnx70,https://www.youtube.com/watch?v=miyE4qxnx70&t=...
1,quando mi è stato chiesto di partecipare a que...,5,13,quando mi è stato chiesto di partecipare a que...,WwxadFn4iV8,https://www.youtube.com/watch?v=WwxadFn4iV8&t=5s
2,sarà necessaria una preparazione quasi formale...,223,241,sarà necessaria una preparazione quasi formale...,ldpgAhQtANs,https://www.youtube.com/watch?v=ldpgAhQtANs&t=...
3,finale molto essenziale molto semplice ma cari...,1018,1030,finale molto essenziale molto semplice ma cari...,u6LDbl4fZMk,https://www.youtube.com/watch?v=u6LDbl4fZMk&t=...
4,la cosa interessante che si può vedere dalle f...,204,215,la cosa interessante che si può vedere dalle f...,yqH9YdpZihk,https://www.youtube.com/watch?v=yqH9YdpZihk&t=...
...,...,...,...,...,...,...
195,poi a seconda della posizione con cui metti la...,54,59,poi a seconda della posizione con cui metti la...,3MyFQsORXtg,https://www.youtube.com/watch?v=3MyFQsORXtg&t=54s
196,per trovare alloggi a breve termine per chi ma...,651,660,per trovare alloggi a breve termine per chi ma...,yeOmbCJpLvE,https://www.youtube.com/watch?v=yeOmbCJpLvE&t=...
197,se stai guardando questo video probabilmente c...,2,8,se stai guardando questo video probabilmente c...,ESB8llTp5nw,https://www.youtube.com/watch?v=ESB8llTp5nw&t=2s
198,in altre ovviamente ha avuto un valore anche e...,146,159,in altre ovviamente ha avuto un valore anche e...,nOdccJajLSI,https://www.youtube.com/watch?v=nOdccJajLSI&t=...


In [14]:
#df_20000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx")
df_20000 = df_20000.head(200)
df_20000

,search_string,start_time,end_time,sentence,video_id,video_url
0,la manifestazione è finita prima del previsto ...,61,63,la manifestazione è finita prima del previsto ...,b5q9uBE5H_s,https://www.youtube.com/watch?v=b5q9uBE5H_s&t=61s
1,finale molto essenziale molto semplice ma cari...,1018,1031,finale molto essenziale molto semplice ma cari...,u6LDbl4fZMk,https://www.youtube.com/watch?v=u6LDbl4fZMk&t=...
2,che stanno proprio nel nostro quartiere verame...,407,420,che stanno proprio nel nostro quartiere verame...,CK71slKyiXU,https://www.youtube.com/watch?v=CK71slKyiXU&t=...
3,ciao cari amici benvenuti nella mia cucina ogg...,0,9,ciao cari amici benvenuti nella mia cucina ogg...,AkW2coApj00,https://www.youtube.com/watch?v=AkW2coApj00&t=0s
4,quello che ci chiedete forse dal primo giorno ...,36,47,quello che ci chiedete forse dal primo giorno ...,9HuoF2bl2us,https://www.youtube.com/watch?v=9HuoF2bl2us&t=36s
...,...,...,...,...,...,...
195,parlando al vento avevamo questa grande teoria...,362,370,parlando al vento avevamo questa grande teoria...,MxQFJFCTFwQ,https://www.youtube.com/watch?v=MxQFJFCTFwQ&t=...
196,ora con un cucchiaio devo fare questo moviment...,279,292,ora con un cucchiaio devo fare questo moviment...,aj9e25TNq7o,https://www.youtube.com/watch?v=aj9e25TNq7o&t=...
197,cera freddo avevamo le mezze maniche però io r...,242,250,cera freddo avevamo le mezze maniche però io r...,nKB1tuL3pR8,https://www.youtube.com/watch?v=nKB1tuL3pR8&t=...
198,continuate a seguirci sulla pagina instagram v...,633,639,continuate a seguirci sulla pagina instagram v...,tc7_0CPeqkY,https://www.youtube.com/watch?v=tc7_0CPeqkY&t=...


In [15]:
#df_40000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx")
df_40000 = df_40000.head(200)
df_40000

,search_string,start_time,end_time,sentence,video_id,video_url
0,si racconta che dopo una discussione sugli aff...,564,569,si racconta che dopo una discussione sugli aff...,2HVKJg__wYk,https://www.youtube.com/watch?v=2HVKJg__wYk&t=...
1,distanza da non essere a rischio ma abbastanza...,549,563,a debita distanza da non essere a rischio ma a...,fomcMM1XA5A,https://www.youtube.com/watch?v=fomcMM1XA5A&t=...
2,quindi ci rendiamo conto che la storia poi va ...,645,668,quindi ci rendiamo conto che la storia poi va ...,dMNIjzeZA1c,https://www.youtube.com/watch?v=dMNIjzeZA1c&t=...
3,consiglio di utilizzare il mix africano per un...,42,46,consiglio di utilizzare il mix africano per un...,AwdMIo8Lv5U,https://www.youtube.com/watch?v=AwdMIo8Lv5U&t=42s
4,la manifestazione è finita prima del previsto ...,61,63,la manifestazione è finita prima del previsto ...,b5q9uBE5H_s,https://www.youtube.com/watch?v=b5q9uBE5H_s&t=61s
...,...,...,...,...,...,...
195,tantino con le patatine hai presente come i ba...,197,205,tantino con le patatine hai presente come i ba...,8NwN0Q55ZuU,https://www.youtube.com/watch?v=8NwN0Q55ZuU&t=...
196,una volta capito bene la frenata con il poster...,270,277,una volta capito bene la frenata con il poster...,iZyMuu3lIu0,https://www.youtube.com/watch?v=iZyMuu3lIu0&t=...
197,perchè un fatto è se ricevete una comunicazion...,148,156,perchè un fatto è se ricevete una comunicazion...,9re4aiaFPVw,https://www.youtube.com/watch?v=9re4aiaFPVw&t=...
198,adesso lo facciamo decollare metteremo il dron...,38,46,adesso lo facciamo decollare metteremo il dron...,UVzPgLEpuQs,https://www.youtube.com/watch?v=UVzPgLEpuQs&t=38s


#### Sending Data To Firestore

##### Language Data

In [16]:
language_data = {"languages":[{"id":1, "name":"English"},{"id":2, "name":"Turkish"},{"id":3, "name":"French"},
                              {"id":4,"name":"Spanish"},{"id":5, "name":"Portuguese"},{"id":6, "name":"Dutch"},
                              {"id":7, "name":"Italian"},{"id":8, "name":"German"},{"id":9, "name":"Arabic"}]}

In [17]:
#fire_db.collection("language-data").document("languages").set(language_data)

##### Language Detail

##### Absolute Beginner

In [18]:
# Absolute Beginner Data
end_time_list = df_28["end_time"].to_list()
video_id_list = df_28["video_id"].to_list()
start_time_list = df_28["start_time"].to_list()

In [19]:
firestore_beginner_data = zip(end_time_list,video_id_list,start_time_list)
beginner_description_text = "description"

In [20]:
beginner_data_list = []
for endtime, videoid, starttime in firestore_beginner_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    beginner_data_list.append(data)
    
#beginner_data_list   

In [21]:
data_beginner = {"description":f"{beginner_description_text}", "level":0, "levelName":"Absolute Beginner", "videos":beginner_data_list}
#data_beginner

In [22]:
#fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_beginner)

##### A1 Level

In [23]:
# A1 Level Data
end_time_list = df_200["end_time"].to_list()
video_id_list = df_200["video_id"].to_list()
start_time_list = df_200["start_time"].to_list()

In [24]:
firestore_A1_data = zip(end_time_list,video_id_list,start_time_list)
A1_description_text = "A1 level description"

In [25]:
A1_data_list = []
for endtime, videoid, starttime in firestore_A1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A1_data_list.append(data)

#A1_data_list   

In [26]:
data_A1 = {"description":f"{A1_description_text}", "level":1, "levelName":"A1", "videos":A1_data_list}
#data_A1

##### A2 Level

In [27]:
# A2 Level Data
end_time_list = df_1000["end_time"].to_list()
video_id_list = df_1000["video_id"].to_list()
start_time_list = df_1000["start_time"].to_list()

In [28]:
firestore_A2_data = zip(end_time_list,video_id_list,start_time_list)
A2_description_text = "A2 level description"

In [29]:
A2_data_list = []
for endtime, videoid, starttime in firestore_A2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A2_data_list.append(data)

#A2_data_list   

In [30]:
data_A2 = {"description":f"{A2_description_text}", "level":2, "levelName":"A2", "videos":A2_data_list}
#data_A2

##### B1 Level

In [31]:
# B1 Level Data
end_time_list = df_5000["end_time"].to_list()
video_id_list = df_5000["video_id"].to_list()
start_time_list = df_5000["start_time"].to_list()

In [32]:
firestore_B1_data = zip(end_time_list,video_id_list,start_time_list)
B1_description_text = "B1 level description"

In [33]:
B1_data_list = []
for endtime, videoid, starttime in firestore_B1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B1_data_list.append(data)

#B1_data_list   

In [34]:
data_B1 = {"description":f"{B1_description_text}", "level":3, "levelName":"B1", "videos":B1_data_list}
#data_B1

##### B2 Level

In [35]:
# B2 Level Data
end_time_list = df_10000["end_time"].to_list()
video_id_list = df_10000["video_id"].to_list()
start_time_list = df_10000["start_time"].to_list()

In [36]:
firestore_B2_data = zip(end_time_list,video_id_list,start_time_list)
B2_description_text = "B2 level description"

In [37]:
B2_data_list = []
for endtime, videoid, starttime in firestore_B2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B2_data_list.append(data)

#B2_data_list   

In [38]:
data_B2 = {"description":f"{B2_description_text}", "level":4, "levelName":"B2", "videos":B2_data_list}
#data_B2

##### C1 Level

In [39]:
# C1 Level Data
video_id_list = df_20000["video_id"].to_list()
end_time_list = df_20000["end_time"].to_list()
start_time_list = df_20000["start_time"].to_list()

In [40]:
firestore_C1_data = zip(end_time_list,video_id_list,start_time_list)
C1_description_text = "C1 level description"

In [41]:
C1_data_list = []
for endtime, videoid, starttime in firestore_C1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C1_data_list.append(data)

#C1_data_list   

In [42]:
data_C1 = {"description":f"{C1_description_text}", "level":5, "levelName":"C1", "videos":C1_data_list}
#data_C1

##### C2 Level

In [43]:
# C2 Level Data
video_id_list = df_40000["video_id"].to_list()
end_time_list = df_40000["end_time"].to_list()
start_time_list = df_40000["start_time"].to_list()

In [44]:
firestore_C2_data = zip(end_time_list,video_id_list,start_time_list)
C2_description_text = "C2 level description"

In [45]:
C2_data_list = []
for endtime, videoid, starttime in firestore_C2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C2_data_list.append(data)

#C2_data_list   

In [46]:
data_C2 = {"description":f"{C2_description_text}", "level":6, "levelName":"C2", "videos":C2_data_list}
#data_C2

##### All Data Sending

In [47]:
data_all = {"datas":[data_beginner, data_A1, data_A2, data_B1, data_B2, data_C1, data_C2]}
#data_all

In [48]:
fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_all)

update_time {
  seconds: 1675441717
  nanos: 954890000
}